<a href="https://colab.research.google.com/github/isabelladegen/ComputationalLogic/blob/prolexa-plus/Tests_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Testing Notebook 
Notebook that makes sure Prolexa continues working


### Installations like demo notebook


In [1]:
!apt-get install swi-prolog -qqq > /dev/null

Extracting templates from packages: 100%


### Installing Prolexa
Same error as on the branch on the forked branch https://github.com/simply-logical/ComputationalLogic/

In [2]:
!yes | pip install git+https://github.com/isabelladegen/ComputationalLogic/ -qqq > /dev/null

  ERROR: Failed building wheel for prolexa
  DEPRECATION: prolexa was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


### Instantiate Prolexa 


In [3]:
from pyswip import Prolog
import prolexa.meta_grammar as meta

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

2021-12-14 11:33:57,256 --------------------------------------------------------------------------------
2021-12-14 11:33:57,258 The model key 'pos' now maps to 'https://huggingface.co/flair/pos-english' on the HuggingFace ModelHub
2021-12-14 11:33:57,260  - The most current version of the model is automatically downloaded from there.
2021-12-14 11:33:57,263  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/pos/en-pos-ontonotes-v0.5.pt)
2021-12-14 11:33:57,265 --------------------------------------------------------------------------------
2021-12-14 11:33:57,327 loading file /root/.flair/models/pos-english/a9a73f6cd878edce8a0fa518db76f441f1cc49c2525b2b4557af278ec2f0659e.121306ea62993d04cd1978398b68396931a39eb47754c8a06a87f325ea70ac63


### Testing existing functionality

Syntax: `test(input_query, expected_answer)`

In [70]:
print("Test 'Tell me about...' queries:\n")
#Unknown noun
test('tell me about unknownnoun', 'I heard you say,  tell me about unknownnoun , could you rephrase that please?')

#Kown noun but nothing is known
test('tell me about tweety', 'I know nothing about tweety')

#Kown noun with knowledge
test('tell me about peter', 'peter is human. peter is mortal')

print("\nTest 'Is...' queries:\n")
#Unknown noun
test('Is unkownnoun human', 'I heard you say,  Is unkownnoun human , could you rephrase that please?')

#Kown noun but nothing is known
test('Is tweety a bird', 'Sorry, I don\'t think this is the case')

#Kown noun with knowledge
test('Is peter mortal', 'peter is mortal')
test('Is peter human', 'peter is human')



Test 'Tell me about...' queries:

Query: tell me about unknownnoun
Answer: I heard you say,  tell me about unknownnoun , could you rephrase that please?

Query: tell me about tweety
Answer: I know nothing about tweety

Query: tell me about peter
Answer: peter is human. peter is mortal


Test 'Is...' queries:

Query: Is unkownnoun human
Answer: I heard you say,  Is unkownnoun human , could you rephrase that please?

Query: Is tweety a bird
Answer: Sorry, I don't think this is the case

Query: Is peter mortal
Answer: peter is mortal

Query: Is peter human
Answer: peter is human



In [58]:
#Helper methods
def query(input):
  answer = meta.standardised_query(pl, input)[0]['Output']
  if isinstance(answer, str):
    return answer
  return str(answer, 'utf-8')

def test(input, expected_answer):
  print(f'Query: {input}')
  actual_answer = query(input)
  assert actual_answer == expected_answer, 'Expected Answer:  ' + expected_answer + '\n Actual answer: ' + actual_answer
  print(f'Answer: {actual_answer}\n')

### Input Method 2
 This method of input is built using a MVP iPywidget and is better for user interaction

In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, HTML

w_textbox = widgets.Textarea(
    value = 'tell me about Peter',
    placeholder='Input text',
    description='Question:',
    layout=Layout(width='900px', height='50px')
)
w_button = widgets.Button(
    description='Ask',
    button_style='info',
    layout=Layout(margin='4px 0px 0px 90px')
)
w_out = widgets.Output(layout={'border': '1px solid black'})


def main(obj):
    query = w_textbox.value.strip()
    answer = meta.standardised_query(pl, query)[0]['Output']
    with w_out:
        print('?', query)
        print(answer)

# bind event handler to UI control
w_button.on_click(main)
# render UI controls in a vertical box
widgets.VBox([w_textbox, w_button, w_out])